In [ ]:
# coleta_e_enriquecimento.py
#
# Análise e Melhorias:
# 1. Otimização da API: A função `obter_dados_verbete` foi refatorada para fazer
#    apenas UMA chamada de API por verbete, em vez de duas. Isso é feito
#    buscando todas as revisões de uma vez e processando a lista em Python
#    para encontrar a primeira e a última, reduzindo significativamente o tempo de execução.
# 2. Melhor Tratamento de Erros: A função agora retorna `None` em caso de falha,
#    tornando o código no loop principal mais limpo e seguro.
# 3. Type Hinting: Foram adicionadas dicas de tipo (ex: `-> list`, `session: requests.Session`)
#    para melhorar a legibilidade e a manutenibilidade do código.
# 4. Uso de f-strings e Pathlib: Modernização leve para melhor clareza.

import requests
import json
import os
import time
from datetime import datetime
from collections import defaultdict
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Any

# --- Configurações Globais ---
WIKI_API_URL = "https://wikifavelas.com.br/api.php"
BASE_DIR = Path('dados')
BASE_DIR.mkdir(exist_ok=True) # Garante que o diretório de saída exista

# --- Funções de Coleta da API ---

def listar_todos_verbetes(session: requests.Session) -> List[Dict[str, Any]]:
    # Lista todos os verbetes da wiki, retornando uma lista de dicionários.
    print("[FASE 1/5] Listando todos os verbetes da wiki...")
    pages = []
    params = {
        'action': 'query', 'format': 'json', 'list': 'allpages',
        'aplimit': 'max', 'continue': ''
    }
    while True:
        try:
            response = session.get(WIKI_API_URL, params=params)
            response.raise_for_status()
            data = response.json()
            batch = data.get('query', {}).get('allpages', [])
            pages.extend(batch)
            cont = data.get('continue')
            if not cont:
                break
            params.update(cont)
        except requests.exceptions.RequestException as e:
            print(f"  [ERRO] Falha ao listar verbetes: {e}. Tentando novamente em 5s...")
            time.sleep(5)
    
    verbetes_info = [{
        'id': p['pageid'],
        'titulo': p['title'],
        'link': f"https://wikifavelas.com.br/{p['title'].replace(' ', '_')}"
    } for p in pages]
    
    print(f"  [INFO] Total de {len(verbetes_info)} verbetes encontrados.")
    return verbetes_info

def obter_dados_verbete_otimizado(page_id: int, session: requests.Session, tentativas: int = 3) -> Optional[Tuple]:
    # Coleta dados detalhados para um único verbete com UMA chamada de API.
    # Retorna None em caso de falha completa.
    params = {
        'action': 'query', 'format': 'json', 'pageids': page_id,
        'prop': 'revisions|links|categories',
        'rvlimit': 'max', 'rvprop': 'user|timestamp',
        'pllimit': 'max', 'cllimit': 'max'
    }
    
    for tentativa in range(1, tentativas + 1):
        try:
            response = session.get(WIKI_API_URL, params=params)
            response.raise_for_status()
            data = response.json()

            page_data = next(iter(data.get('query', {}).get('pages', {}).values()), None)
            
            if not page_data or 'revisions' not in page_data:
                return {}, [], [], "N/A", "N/A", "N/A"

            revisoes = page_data.get('revisions', [])
            
            # Processa edições
            usuarios = defaultdict(int)
            for rev in revisoes:
                usuarios[rev.get('user', 'Desconhecido')] += 1

            # Processa referências e categorias
            referencias = [link['title'] for link in page_data.get('links', [])]
            categorias = [cat['title'] for cat in page_data.get('categories', [])]
            
            # Encontra primeira e última edição na lista de revisões
            # A API retorna a mais recente primeiro por padrão
            data_ultima_edicao = revisoes[0].get('timestamp', 'N/A')
            autor_criacao = revisoes[-1].get('user', 'N/A')
            data_criacao = revisoes[-1].get('timestamp', 'N/A')

            return dict(usuarios), referencias, categorias, autor_criacao, data_criacao, data_ultima_edicao

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"  [ERRO] Tentativa {tentativa} para page_id {page_id} falhou: {e}. Aguardando...")
            time.sleep(2 * tentativa)

    print(f"  [FALHA] Todas as tentativas para page_id {page_id} falharam.")
    return None

# --- Script Principal ---

def main() -> None:
    session = requests.Session()
    verbetes_base = listar_todos_verbetes(session)
    
    verbetes_enriquecidos = []
    total_verbetes = len(verbetes_base)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_filename = f'dados_api_{timestamp}.json'
    output_path = BASE_DIR / output_filename

    print("\n[FASE 2 a 5] Enriquecendo cada verbete com dados adicionais...")
    for i, verbete in enumerate(verbetes_base, start=1):
        print(f"  Processando {i}/{total_verbetes}: \"{verbete['titulo']}\"")
        
        dados_coletados = obter_dados_verbete_otimizado(verbete['id'], session)
        
        verbete_completo = verbete.copy()
        
        if dados_coletados:
            usuarios, refs, cats, autor, criacao, ultima = dados_coletados
            verbete_completo.update({
                'quantidade_edicoes': sum(usuarios.values()),
                'usuarios_edicoes': usuarios,
                'referencias': refs,
                'categorias': cats,
                'autor_criacao': autor,
                'data_criacao': criacao,
                'data_ultima_edicao': ultima
            })
        else: # Se a coleta falhou, preenche com dados vazios
             verbete_completo.update({
                'quantidade_edicoes': 0, 'usuarios_edicoes': {}, 'referencias': [],
                'categorias': [], 'autor_criacao': "Falha", 'data_criacao': "Falha",
                'data_ultima_edicao': "Falha"
            })

        verbetes_enriquecidos.append(verbete_completo)
        
        if i % 50 == 0 or i == total_verbetes:
            dados_para_salvar = {
                'total_verbetes': total_verbetes,
                'verbetes_completo': verbetes_enriquecidos
            }
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(dados_para_salvar, f, ensure_ascii=False, indent=2)
            print(f"  [SALVO] Progresso salvo em '{output_path}' ({i} verbetes)")

    print(f"\n[FINALIZADO] Processo de coleta e enriquecimento concluído.")
    print(f"Arquivo final salvo em: '{output_path}'")

if __name__ == '__main__':
    main()


[FASE 1/5] Listando todos os verbetes da wiki...
  [INFO] Total de 3552 verbetes encontrados.

[FASE 2 a 5] Enriquecendo cada verbete com dados adicionais...
  Processando 1/3552: ""A Expansão das Milícias no Rio de Janeiro: uso da força estatal, mercado imobiliário e grupos armados" (Resenha)"
  Processando 2/3552: ""Bairro Veneza – Balsas - MA: Dados e Aspectos Socioeconômicos""
  Processando 3/3552: ""Estudo sobre a distribuição das taxas de encarceramento nos estados brasileiros e principais variáveis associadas: Influências socioeconômicas e ideológicas (Resenha)""
  Processando 4/3552: ""LUA"-Sem luta não tem vitória"
  Processando 5/3552: ""O Sol na Cabeça" e a Realidade das Favelas do Rio de Janeiro"
  Processando 6/3552: ""Security is everyone's duty": collective representations and new networks of post-pacification security policies in Rio de Janeiro (article)"
  Processando 7/3552: ""Segurança é um dever de todos": representações coletivas e novas redes de políticas de segur